### Load libraries

In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import tqdm
import sys
import numpy as np
import pandas as pd

### Load helpers

In [6]:
# This may need to change
sys.path.insert(0, "/Users/harrisonchase/workplace/sports/")

from clean_sports_work.sports_reference.api import find_table, extract_table, create_insert_table_sql

### Get data for each year

In [11]:
all_dfs = []
for year in tqdm.tqdm(range(1950, 2020)):
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    table_str = find_table(soup, 'advanced_stats')

    yr2018 = extract_table(table_str, header_row=0, get_url=True, start_of_rows=1)
    drop_cols = yr2018.isnull().mean()[lambda x: x == 1].index
    for col in drop_cols:
        del yr2018[col]
    yr2018['year'] = year
    all_dfs.append(yr2018)

100%|██████████| 70/70 [03:47<00:00,  5.48s/it]


In [12]:
all_draft_dfs = []
for year in tqdm.tqdm(range(1950, 2020)):
    url = 'https://www.basketball-reference.com/draft/NBA_{}.html'.format(year)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    table_str = find_table(soup, 'stats')

    yr2018 = extract_table(table_str, header_row=1, get_url=True, start_of_rows=2)
    drop_cols = yr2018.isnull().mean()[lambda x: x == 1].index
    for col in drop_cols:
        del yr2018[col]
    yr2018['year'] = year
    all_draft_dfs.append(yr2018)

100%|██████████| 70/70 [02:05<00:00,  1.34s/it]


### Rough cleaning of data

In [49]:
all_stats = pd.concat(all_dfs)
all_stats = all_stats.replace('', np.nan)
all_stats = all_stats.dropna(subset=['player_url', 'mp', 'age'])
float_cols = ['bpm', 'ts_pct', 'per', 'usg_pct', 'obpm', 'dbpm', 
              'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct',
             'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'ws', 'ows', 'dws']
for col in float_cols:
    all_stats[col] = all_stats[col].astype(float)
    
int_cols = ['mp', 'age']
for col in int_cols:
    all_stats[col] = all_stats[col].astype(int)

In [50]:
all_stats['pos'] = all_stats['pos'].str.split('-').str[0]

### Draft stats

In [51]:
all_draft = pd.concat(all_draft_dfs).dropna(subset=['player_url'])
all_draft = all_draft.replace('', np.nan)

In [52]:
all_draft['pick_overall'] = all_draft['pick_overall'].astype(float)

In [53]:
all_stats = all_stats.merge(all_draft[['player_url', 'pick_overall', 'college_name']], how='left', on='player_url')

### Create id for each (player, year)

In [54]:
all_stats['id'] = all_stats['player_url'] + '___' + all_stats['year'].astype(str)

### Deduplicate ids

Occurs if player played on multiple teams in one season, take row where team == 'TOT'

In [55]:
multiple_ids = all_stats['id'].value_counts()[lambda x: x> 1].index

In [67]:
changed = all_stats[all_stats['id'].isin(multiple_ids) & (all_stats['team_id'] == 'TOT')]
first_team = all_stats.iloc[changed.index + 1]
if not all(first_team['player_url'].values == changed['player_url'].values):
    raise ValueError
changed['started_team'] = first_team['team_id'].values
no_change = all_stats[~all_stats['id'].isin(multiple_ids)]
no_change['started_team'] = no_change['team_id']
base_all_stats = pd.concat([
    no_change,
    changed,
])

/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Add in row for next year

Be careful not to use this in your model evaluation!!! Is only present for ease of predicting next year

In [76]:
most_recent_year = base_all_stats[base_all_stats['year'] == base_all_stats['year'].max()]
most_recent_year['year'] += 1
most_recent_year['age'] +=1
most_recent_year['id'] = most_recent_year['player_url'] + '___' + most_recent_year['year'].astype(str)


/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on

In [112]:
def get_team(url):
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    team_p = [x for x in soup.findAll('div', {'id': 'meta'})[0].findAll('p') if x.find('strong') and (x.find('strong').text == 'Team')]

    if len(team_p) == 0:
        return np.nan
    elif len(team_p) == 1:
        return team_p[0].find('a')['href'][7:10]
    else:
        raise ValueError

In [116]:
teams_now = []
for url_suffix in tqdm.tqdm(most_recent_year['player_url']):
    url = 'https://www.basketball-reference.com' + url_suffix
    teams_now.append(get_team(url))


 29%|██▉       | 155/530 [03:22<09:24,  1.51s/it]


 58%|█████▊    | 309/530 [06:46<04:36,  1.25s/it]


 87%|████████▋ | 463/530 [10:25<02:51,  2.55s/it]


100%|██████████| 530/530 [12:04<00:00,  1.28s/it]


In [119]:
most_recent_year['team_id'] = teams_now
most_recent_year['started_team'] = teams_now

/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [122]:
most_recent_year = most_recent_year.dropna(subset=['started_team'])

In [123]:
for col in most_recent_year.columns:
    if col not in ['player', 'pos', 'age', 'team_id', 'id', 'player_url', 'pick_overall', 'college_name', 'started_team', 'year']:
        most_recent_year[col] = np.nan

/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [125]:
all_stats = pd.concat([base_all_stats, most_recent_year])

### Save data

In [126]:
all_stats.to_msgpack('all_stats.mp')